# Import Libraries

Install keras visualize library 

In [ ]:
!pip install git+https://github.com/raghakot/keras-vis.git

In [ ]:
import tensorflow as tf
import keras
from keras import models
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.densenet import DenseNet121
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from vis.utils import utils
import itertools
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import PIL
import cv2

# Access Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Dataset Path

In [ ]:
train_path = ''
val_path = ''
test_path = ''

# Parameters

In [ ]:
img_size = 224
epochs = 5
batch_size = 50

# Normalize Dataset from 0-255 to 0-1

In [ ]:
train_norm = ImageDataGenerator(rescale=1. / 255) 
val_norm = ImageDataGenerator(rescale=1. / 255) 
test_norm = ImageDataGenerator(rescale=1. / 255) 

In [ ]:
train_generator = train_norm.flow_from_directory( 
    train_path, 
    target_size=(img_size, img_size), 
    batch_size=batch_size)

val_generator = val_norm.flow_from_directory( 
    val_path, 
    target_size=(img_size, img_size), 
    batch_size=batch_size)

test_generator = test_norm.flow_from_directory( 
    test_path, 
    target_size=(img_size, img_size), 
    batch_size=batch_size,
    shuffle=False)

In [ ]:
num_train_files = len(train_generator.filenames)
num_train_classes = len(train_generator.class_indices)
print('num_train_files', num_train_files, '&', 'num_train_classes', num_train_classes )

num_val_files = len(val_generator.filenames)
num_val_classes = len(val_generator.class_indices)
print('num_val_files', num_val_files, '&', 'num_val_classes', num_val_classes )

num_test_files = len(test_generator.filenames)
num_test_classes = len(test_generator.class_indices)
print('num_test_files', num_test_files, '&', 'num_test_classes', num_test_classes )

In [ ]:
train_steps = np.ceil(num_train_files/batch_size)
val_steps = np.ceil(num_val_files/batch_size)

# Build Model

In [ ]:
model = DenseNet121(weights='imagenet')

In [ ]:
x = model.layers[-2].output
predictions = Dense(num_train_classes, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(lr=0.01), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

Save Model

In [ ]:
model_path = '/densenet121_model.h5'

In [ ]:
checkpoint = ModelCheckpoint(model_path, monitor='val_acc', verbose=1,
                             save_best_only=True, mode='max')

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2,
                              verbose=1, mode='max', min_lr=0.00001)

In [ ]:
log_dir = '/tf-log/'
tb_cb = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

In [ ]:
cb_list = [checkpoint, reduce_lr, tb_cb]

In [ ]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=train_steps,
                              validation_data=val_generator,
                              validation_steps=val_steps,
                              epochs=epochs,
                              verbose=1,
                              callbacks=cb_list)

Save Model Weight

In [ ]:
model.save_weights('weights.h5')

In [ ]:
_img = load_img('',target_size=(224,224))
plt.imshow(_img)
plt.show()

In [ ]:
# Utility to search for layer index by name. 
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = utils.find_layer_idx(model, 'dense_1')
# Swap softmax with linear
model.layers[layer_idx].activation = keras.activations.linear
model = utils.apply_modifications(model)

In [ ]:
img_path = '.jpg'
img=mpimg.imread(img_path)
plt.imshow(img)

In [ ]:
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
import numpy as np
x = np.expand_dims(x, axis=0)